In [15]:
import requests
import json
import pandas as pd
import numpy as np
import jsonlines
import re
import pprint
import pandas as pd

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

> 한겨레 전체 크롤링

In [82]:
raw_df =pd.read_excel("/Users/ijiseon/Desktop/한겨레_20230513-20240513.xlsx")

# 필요한 칼럼만 선택
raw_df = raw_df[['일자', '언론사', '제목', 'URL']]

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [84]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# WebDriver 설정 (Chrome 사용)
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

# 기사 내용을 저장할 리스트
news_contents = []

# WebDriver가 페이지의 요소를 로드할 때까지 최대 10초간 기다리도록 설정
wait = WebDriverWait(driver, 10)

for url in raw_df['URL']:
    try:
        driver.get(url)
        # 지정된 요소가 DOM에 존재하고 보이게 될 때까지 기다립니다.
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.article-text p.text')))

        # BeautifulSoup으로 페이지의 HTML 분석
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        news_paragraphs = soup.select('div.article-text p.text')
        clean_text = ' '.join([p.get_text(strip=True) for p in news_paragraphs])

        # 리스트에 기사 내용 추가
        news_contents.append(clean_text if clean_text else '내용 없음')
    except Exception as e:
        print(f"Error fetching {url}: {str(e)}")
        news_contents.append('크롤링 실패')

driver.quit()  # WebDriver 종료

# DataFrame 업데이트
raw_df['기사내용'] = news_contents



Error fetching http://www.hani.co.kr/arti/politics/defense/1139006.html: Message: 
Stacktrace:
0   chromedriver                        0x0000000108df6a78 chromedriver + 5216888
1   chromedriver                        0x0000000108dee6b3 chromedriver + 5183155
2   chromedriver                        0x0000000108960527 chromedriver + 406823
3   chromedriver                        0x00000001089abff2 chromedriver + 716786
4   chromedriver                        0x00000001089ac181 chromedriver + 717185
5   chromedriver                        0x00000001089f01d4 chromedriver + 995796
6   chromedriver                        0x00000001089ce72d chromedriver + 857901
7   chromedriver                        0x00000001089ed6bf chromedriver + 984767
8   chromedriver                        0x00000001089ce4a3 chromedriver + 857251
9   chromedriver                        0x000000010899efe3 chromedriver + 663523
10  chromedriver                        0x000000010899f92e chromedriver + 665902
11  chromedr

In [87]:
raw_df.head(10)

,일자,언론사,제목,URL,기사내용
0,20240513,한겨레,"윤 대통령, 저출생수석실 설치 준비 지시",http://www.hani.co.kr/arti/politics/politics_g...,윤석열 대통령이 13일 “저출생수석실 설치를 준비하라”고 지시했다. 윤 대통령은 이...
1,20240513,한겨레,"최상목 “내년 예산, 저출산 청년 등 부처 협업예산 강화”",http://www.hani.co.kr/arti/economy/economy_gen...,최상목 부총리 겸 기획재정부 장관이 “내년도 정부예산은 범부처 협업을 대폭 강화해 ...
2,20240513,한겨레,박찬대 “라인사태 총력 대응 안 하면 매국 정부 비판받을 것”,http://www.hani.co.kr/arti/politics/politics_g...,박찬대 더불어민주당 원내대표가 13일 ‘라인야후 사태’ 관련 정부의 대응을 두고 “...
3,20240513,한겨레,유승민 “일본이 네이버 지분 빼앗는데 정부가 못 막고 있어”,http://www.hani.co.kr/arti/politics/politics_g...,유승민 전 국민의힘 의원이 13일 일본 정부의 행정 지도가 촉발한 라인 야후 지분 ...
4,20240513,한겨레,"푸틴, 측근 쇼이구 국방장관 경질 경제전문가 발탁한 이유는?",http://www.hani.co.kr/arti/international/europ...,블라디미르 푸틴 러시아 대통령이 오랜 측근인 세르게이 쇼이구 국방장관을 경질하고 정...
5,20240513,한겨레,김용민 “채상병 특검 거부권 행사는 위헌적 탄핵 사유 될 수 있어”,http://www.hani.co.kr/arti/politics/politics_g...,더불어민주당 원내정책 수석부대표인 김용민 의원이 해병대 채 상병 순직 사건 수사 외...
6,20240513,한겨레,라인 사태가 ‘반일 몰이’인가 [5월13일 뉴스뷰리핑],http://www.hani.co.kr/arti/society/society_gen...,"정치, 경제, 사회, 국제 분야를 두루 취재하고 워싱턴 특파원을 지낸 권태호 논설실..."
7,20240513,한겨레,"한러 관계, 저절로 회복될까 [문정인 칼럼]",http://www.hani.co.kr/arti/opinion/column/1140...,우크라이나 전쟁이 종료되면 한-러 관계 또한 복원할 수 있으리라는 기대는 지나치게 ...
8,20240513,한겨레,선방위 152일의 폭주 윤 부부 정부여당 비판 보도에 제재 28건,http://www.hani.co.kr/arti/society/media/11402...,22대 국회의원선거방송심의위원회(선방위)가 지난 10일 임기를 마쳤다. 백선기 위원...
9,20240513,한겨레,선방위 접수한 ‘넘버3’ 강경파 “충성 인정받으려 요란한 칼질”,http://www.hani.co.kr/arti/society/media/11402...,지난달 29일 전국언론노동조합(언론노조)은 22대 국회의원선거방송심의위원회(선방위)...


In [88]:
raw_df.to_excel("hani.xlsx")

/var/folders/f4/skyf3wfx5570p30yjkrfkxd00000gn/T/ipykernel_1299/3481350682.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  raw_df.to_excel("hani.xlsx")


In [92]:
raw_df["URL"][0]

'http://www.hani.co.kr/arti/politics/politics_general/1140306.html'

In [90]:
raw_df["기사내용"][0]

'윤석열 대통령이 13일 “저출생수석실 설치를 준비하라”고 지시했다. 윤 대통령은 이날 오전 수석비서관 회의를 주재한 자리에서 이같이 말했다고 김수경 대통령실 대변인이 서면 브리핑을 통해 전했다. 윤 대통령은 지난 9일 취임 2주년 기자회견에서 ‘저출생대응기획부(가칭)’를 신설하겠다고 밝힌 바 있다. 윤 대통령은 당시 “저출생대응기획부 장관이 사회부총리를 맡도록 해서 교육·노동·복지를 아우르는 정책을 수립하고, 단순한 복지정책 차원을 넘어 국가 어젠다가 되도록 하겠다”고 말했다. ‘대통령실 슬림화’를 약속하며 2실(비서실·안보실) 5수석(정무·홍보·시민사회·경제·사회) 체제로 출범한 대통령실은, 윤 대통령 집권 2년을 맞아 ‘3실(비서실·정책실·안보실) 7수석(정무·홍보·민정·시민사회·경제·사회·과학기술)’체제로 덩치가 커진 상태다. 저출생수석실을 설치할 경우엔 3실 8수석 체제가 된다. 이승준 기자gamja@hani.co.kr'

In [94]:
raw_df["URL"][1]

'http://www.hani.co.kr/arti/economy/economy_general/1140284.html'

In [91]:
raw_df["기사내용"][1]

"최상목 부총리 겸 기획재정부 장관이 “내년도 정부예산은 범부처 협업을 대폭 강화해 개별부처가 아닌 국민의 입장에서 편성하겠다”고 밝혔다. 최 부총리는 13일 오전 정부서울청사에서 열린 경제장관회의에 참석해 “저출생 대응, 청년지원, 첨단·전략산업 인력양성 등 주요 투자과제 중심으로 부처 간 연계를 강화하겠다”며 이같이 말했다. 최 부총리가 언급한 것은 ‘협업예산'이다. 각 정부 부처 사이 쳐진 예산 편성의 칸막이를 넘어 특정 주제를 중심으로 범부처 예산을 재구조화하는 작업이다. 부처별로 진행되는 사업 가운데 같은 정책적 목표를 지닌 사업을 한 지붕 아래로 모아 검토하면서 재정 운용의 효율성을 높이겠다는 것이다. 이어 최 부총리는 “민생문제 해결과 단단한 중산층 구축, 우리 경제 역동성 제고와 양질의 일자리 창출 등 경제팀 앞에 놓인 숙제가 여전히 많다”며 “복합 이슈일수록 부처 간 칸막이를 허물어 정책 공간과 정책 여력을 넓혀 반드시 해답을 찾는 경제원팀이 되겠다”고 말했다. 이날 최 부총리는 지역 창업생태계 성장을 위한 ‘지역 성장지원 서비스 경쟁력 강화방안’도 발표했다. 기술창업기업의 약 60%, 벤처투자의 약 80%가 수도권에 집중된 쏠림 현상을 줄이겠다는 목표다. 먼저 지역 벤처투자 생태계의 마중물 역할을 하는 모태펀드 지역계정 출자규모를 올해 1천억원에서 단계적으로 확대해 오는 2026년까지 누적 1조원 이상 신규 공급한다. 민간 출자자의 부담을 줄일 수 있도록 지방자치단체가 개인투자조합에 20% 이상 출자하는 경우 지자체와 법인의 출자 상한선을 현행 30%에서 49%까지 올리고, 수도권보다 열악한 법률·회계·컨설팅·디자인·지식재산권 등 전문서비스 지원도 강화한다. 한편, 기재부는 그간 ‘비상경제장관회의’로 운영된 회의 명칭을 이날부터 ‘경제관계장관회의’로 전환했다. 전임 추경호 부총리가 2022년 6월19일 첫 비상경제장관회의를 연 뒤 약 2년간 지속한 회의 명칭을 바꾼 것이다. 기재부 관계자는 “복합위기라고 부를 만큼 거시 지표가 나쁘지 않고, 